In [1]:
!nvidia-smi

Mon Jun 24 23:47:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install pypdf

In [3]:
!pip install -q transformers einops accelerate bitsandbytes langchain

In [4]:
!pip install llama-index==0.9.39

In [5]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [6]:
!mkdir Data



mkdir: cannot create directory ‘Data’: File exists


In [7]:
docu = SimpleDirectoryReader('/content/Data').load_data()

In [8]:
docu

[Document(id_='4266ccda-7dd6-461a-bc36-a4598e8fdde2', embedding=None, metadata={'page_label': '1', 'file_name': 'Document.pdf', 'file_path': '/content/Data/Document.pdf', 'file_type': 'application/pdf', 'file_size': 147396, 'creation_date': '2024-06-24', 'last_modified_date': '2024-06-24', 'last_accessed_date': '2024-06-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="   \n \n   \n Shashank Kurakula  \nCharlotte, North Carolina 28213  | (903) -468-2965  | LinkedIn  | Github  | Portfolio  \nProfile  \nExperienced Data Engineer adept at designing and implementing robust data pipelines, optimizing  data warehouses, \nand ensuring efficient ETL processes. Proficient in Python, skilled in deploying and managing cloud solutions using \nAzure technologies,

In [9]:
system_promt = """
You are a helping answer my interview questions.
Your goal is to answer questions as lively as possible based on the instructions and context provided.
"""

query_wrapper_prompt = SimpleInputPrompt("<|User|>{query_str}<|Assistant|>")

In [10]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGr

In [11]:
# Loading Mistral model

import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs= {"temperature":0.0, "do_sample":False},
    system_prompt= system_promt,
    query_wrapper_prompt= query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.3",
    model_name="mistralai/Mistral-7B-Instruct-v0.3",
    device_map="auto",
    model_kwargs= {"torch_dtype": torch.float16, "load_in_8bit":True} # Enable CPU offloading for 8-bit quantization
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
!pip install -U langchain-community

In [13]:
!pip install sentence_transformers

In [14]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

In [15]:
embed_model = LangchainEmbedding(
    HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model,
)

In [17]:
index = VectorStoreIndex.from_documents(docu, service_context=service_context)

In [18]:
query_engine = index.as_query_engine()

In [20]:
query = "Tell me about Your self"
response = query_engine.query(query)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I am Shashank Kurakula, an experienced Data Engineer currently residing in Charlotte, North Carolina. I have a Master's degree in Computer and Information Science from Texas A&M University - Commerce, which I will be graduating in December 2022. Prior to that, I completed my Bachelor's degree in Computer Science and Engineering from Jawaharlal Nehru Technological University.

I specialize in designing and implementing robust data pipelines, optimizing data warehouses, and ensuring efficient ETL processes. I am proficient in Python and skilled in deploying and managing cloud solutions using Azure technologies, including Azure Data Lake for scalable storage solutions, and Data Factory for streamlined data integration.

I have experience working as a Junior Data Engineer at Rudrakshx, where I designed and implemented data pipelines using Data Factory, resulting in a 30% reduction in data processing time. I also developed and fine-tuned ETL workflows, achieved streamlined data integration,